### Векторные представления слов (word embeddings)

#### word2vec

Мера семантической близости — мера близости, предназначенная для количественной оценки семантической схожести слов. Такая мера показывает высокие значения для пар слов, находящихся в семантических отношениях (синонимия, ассоциативность и т.д.), и нулевые значения для всех остальных пар.

word2vec - алгоритм для получения векторных представлений слов. Подход основан на важной гипотезе, которую в науке принято называть гипотезой локальности — “слова, которые встречаются в одинаковых окружениях, имеют близкие значения”. Близость в данном случае понимается очень широко, как то, что рядом могут стоять только сочетающиеся слова. Например, для нас привычно словосочетание "заводной будильник". А сказать “заводной апельсин” мы не можем* — эти слова не сочетаются.

##### Алгоритм word2vec
Мы будем предсказывать вероятность слова по его окружению (контексту). То есть мы будем учить такие вектора слов, чтобы вероятность, присваиваемая моделью слову была близка к вероятности встретить это слово в этом окружении в реальном тексте.





<img src='image/w2v_formula.PNG'>

Здесь W0 — вектор целевого слова, Wc — это некоторый вектор контекста, вычисленный (например, путем усреднения) из векторов окружающих нужное слово других слов. А S — это функция, которая двум векторам сопоставляет одно число. Например, это может быть косинусное расстояние.

Процесс тренировки устроен следующим образом: мы берем последовательно (2k+1) слов, слово в центре является тем словом, которое должно быть предсказано. А окружающие слова являются контекстом длины по k с каждой стороны. Каждому слову в нашей модели сопоставлен уникальный вектор, который мы меняем в процессе обучения нашей модели. В целом, этот подход называется CBOW — continuous bag of words, continuous потому, что мы скармливаем нашей модели последовательно наборы слов из текста, a BoW потому что порядок слов в контексте не важен.
<img src='image/CBOW_.png'>
Другой подход skip-gram — прямо противоположный CBOW, то есть “словосочетание с пропуском”. Мы пытаемся из данного нам слова угадать его контекст (точнее вектор контекста). В остальном модель не претерпевает изменений.
<img src='image/skipgram.png'>

Что стоит отметить: хотя в модель не заложено явно никакой семантики, а только статистические свойства корпусов текстов, оказывается, что натренированная модель word2vec может улавливать некоторые семантические свойства слов. Классический пример:

<img src='image/word_embeddings.PNG'>

Слово "мужчина" относится к слову "женщина" так же, как слово "дядя" к слову "тётя", что для нас совершенно естественно и понятно, но в других моделям добиться такого же соотношения векторов можно только с помощью специальных ухищрений. Здесь же — это происходит естественно из самого корпуса текстов.

In [1]:
import gensim.downloader as api

In [2]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [3]:
word_vectors

NameError: name 'word_vectors' is not defined

In [ ]:
import gensim.downloader as api

word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

In [ ]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

print(word_vectors.doesnt_match("black green summer brown".split()))

In [ ]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

In [ ]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

# Simple chat-bot example

In [13]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [12]:
!pip install annoy

In [14]:
import pandas as pd

In [15]:
!ls

lesson_3.ipynb         negative.csv           translate_example
lesson_3_exemple.ipynb positive.csv


In [16]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-12-18 11:20:33--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-12-18 11:20:33--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2022-12-18 11:20:34 ERROR 404: Not Found.

--2022-12-18 11:20:34--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2022-12-18 11:20:34--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connec

In [17]:
neg = pd.read_csv('negative.csv', sep=';')
pos = pd.read_csv('positive.csv', sep=';')

In [18]:
tweets = pos.iloc[:, 3]
tweets = tweets.append(neg.iloc[:, 3])
tweets.head()

/var/folders/n3/rc0k_vzs5ln66jll2sd1t7wh0000gn/T/ipykernel_59927/4187563560.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets = tweets.append(neg.iloc[:, 3])


0    Да, все-таки он немного похож на него. Но мой ...
1    RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
2    RT @digger2912: "Кто то в углу сидит и погибае...
3    @irina_dyshkant Вот что значит страшилка :D\nН...
4    ну любишь или нет? — Я не знаю кто ты бля:D ht...
dtype: object

In [19]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [20]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

In [21]:
%%time
tweets = tweets.apply(lambda x: preprocess_txt(x))

KeyboardInterrupt: 

In [22]:
tweets

0         Да, все-таки он немного похож на него. Но мой ...
1         RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
2         RT @digger2912: "Кто то в углу сидит и погибае...
3         @irina_dyshkant Вот что значит страшилка :D\nН...
4         ну любишь или нет? — Я не знаю кто ты бля:D ht...
                                ...                        
111917    Но не каждый хочет что то исправлять:( http://...
111918    скучаю так :-( только @taaannyaaa вправляет мо...
111919            Вот и в школу, в говно это идти уже надо(
111920    RT @_Them__: @LisaBeroud Тауриэль, не грусти :...
111921    Такси везет меня на работу. Раздумываю приплат...
Length: 226832, dtype: object

In [27]:
Word2Vec?

In [29]:
modelW2V = Word2Vec(sentences=tweets, vector_size=300, window=5, min_count=3)

In [31]:
modelFT = FastText(sentences=tweets, vector_size=300, min_count=3, window=5, workers=8)

In [52]:
%%time
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0


for tweet in tweets[:100000]:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = tweet
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in tweet:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

w2v_index.build(10)
ft_index.build(10)

CPU times: user 37.7 s, sys: 1min 15s, total: 1min 53s
Wall time: 1h 3min 7s


True

In [53]:
def get_response(tweet, index, model, index_map):
    tweet = preprocess_txt(tweet)
    print(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [54]:
TEXT = "какой город самы красивый"

In [55]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['город', 'красивый']


[['красивый', 'зверь', 'мечта', 'httptco7ao4xpgdda'],
 ['глебушко',
  'рождение',
  'рость',
  'больший',
  'здоровый',
  'красивый',
  'маленький',
  'эгоист',
  'сестра'],
 ['staasya08', 'поздравлять', 'растить', 'больший', 'красивый', 'здоровый']]

In [56]:
get_response(TEXT, ft_index, modelFT, index_map)

['город', 'красивый']


[['город', 'красивосолнышкоснег', 'белыйнебо', 'чистый'],
 ['снег', 'смешной', 'жалкий', 'скудность', 'свой'],
 ['таки',
  'прекрасный',
  'погода',
  'мир',
  'красивый',
  'чистый',
  'белый',
  'снег']]

Обе модели мправились хорошо, интересно было бы посмотреть на результат если в других твитах 
не было слово "красивый". Но в данном примере результат на лицо